<a href="https://colab.research.google.com/github/factorLee/DeepLearningfromScratch01_PKNU/blob/main/AI_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 201730262 이상주 인공지능개론 과제#3

In [ ]:
!unzip ai.zip

unzip:  cannot find or open ai.zip, ai.zip.zip or ai.zip.ZIP.


In [ ]:
from tensorflow import keras
fashion_mnist = keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train/255.0
x_test = x_test/255.0

4423680/4422102 [==============================] - 0s 0us/step


## 데이터 차원 확인

In [ ]:
print(x_train.shape) # 28X28 이미지 6만개
print(x_train[0].shape) # 28X28 이미지 확인
print(y_train.shape) # 패션물품 라벨 6만개
print(y_train[0].shape) # scalar

(60000, 28, 28)
(28, 28)
(60000,)
()


In [ ]:
# 테스트 데이터 확인
print(x_test.shape)
print(y_test.shape)

(10000, 28, 28)
(10000,)


In [ ]:
from common.layers import *
from common.gradient import *
from collections import OrderedDict #순서를 기억하는 딕셔너리

ModuleNotFoundError: ignored

## Layer

In [ ]:
import numpy as np

class ThreeLayerNet:
    def __init__(self, input_size, hidden01_size, hidden02_size, output_size,
                 weight_init_std=0.01):
        
        # 패러미터 초기화
        self.params={}
        self.params['W1'] = weight_init_std*np.random.randn(input_size,hidden01_size)
        self.params['b1'] = np.zeros(hidden01_size)
        self.params['W2'] = weight_init_std*np.random.randn(hidden01_size,hidden02_size)
        self.params['b2'] = np.zeros(hidden02_size)
        self.params['W3'] = weight_init_std * np.random.randn(hidden02_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        

        # 계산할 layer를 추가
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'],self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2']= Affine(self.params['W2'], self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine3']= Affine(self.params['W3'], self.params['b3'])
        self.lastLayer = SoftmaxWithLoss()

    # 신경망의 정방향 계산
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    # 주어진 x에 대하여 정답 y와 비교하여 Loss Function의 값을 계산
    def loss(self,x,y):
        y_pred = self.predict(x)
        return self.lastLayer.forward(y_pred, y)

    # 정확도 계산
    def accuracy(self, x, y):
        y_pred = self.predict(x)
        y_pred = np.argmax(y_pred,axis=1)
        if y.ndim != 1:
            y = np.argmax(y,axis=1)
        accuracy = np.sum(y_pred == y)/ float(x.shape[0])
        return accuracy

    # 기울기 계산
    def gradient(self, x, y):
        self.loss(x,y)

        dout = 1
        dout = self.lastLayer.backward(dout)
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        grads['W3'] = self.layers['Affine3'].dW
        grads['b3'] = self.layers['Affine3'].db
        return grads

## 학습

In [ ]:
from mnist import load_mnist

(x_train,y_train),(x_test,y_test) = load_mnist(normalize=True, one_hot_label=True)

network =ThreeLayerNet(input_size=784, hidden01_size=50, hidden02_size=100, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list=[]
train_acc_list=[]
test_acc_list=[]

iter_per_epoch = max(train_size/batch_size,1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    y_batch = y_train[batch_mask]

    # 주어진 배치에서 기울기 계산
    grad = network.gradient(x_batch,y_batch)

    # 기울기에 따라 패러미터 갱신
    for key in ['W1','b1','W2','b2','W3','b3']:
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss)
    
    if i%iter_per_epoch == 0:
        train_acc = network.accuracy(x_train,y_train)
        test_acc = network.accuracy(x_test,y_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

------------------------------------------